# 실습: Hugging Face로 모델/토크나이저 로딩·추론
**목표**
- `AutoTokenizer/AutoModel*` & `pipeline` 기본 사용법 익히기
- BERT(인코더) / GPT-2(디코더) 계열 모델을 불러와 간단 추론
- 파인튜닝된 모델과 범용 모델을 **태스크별로** 비교 실행
- 태스크: 텍스트 분류 · 질의응답(Extractive QA) · 번역(KO↔EN) · 텍스트 생성


## 1) 환경 준비 (설치)

In [1]:
%pip -q install transformers>=4.42.0 datasets>=2.17 accelerate>=0.31 sentencepiece safetensors

## 2) 공통 임포트 & 디바이스 확인

In [2]:
import torch, platform
from transformers import (
    AutoTokenizer, AutoModel, AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering, AutoModelForCausalLM,
    pipeline
)
print("PyTorch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("Python:", platform.python_version())

PyTorch: 2.8.0+cu126 | CUDA: False
Python: 3.12.11


## 3) 모델 ID 설정 (필요 시 교체)

In [9]:
MODEL_CLS = "nlptown/bert-base-multilingual-uncased-sentiment"  # 다국어 감성 (별점 1~5)
MODEL_QA  = "deepset/xlm-roberta-base-squad2"                   # 다국어 QA
MODEL_MT_KOEN = "Helsinki-NLP/opus-mt-ko-en"                    # 번역 KO→EN
MODEL_MT_ENKO = "facebook/nllb-200-distilled-600M"              # 번역 EN→KO
MODEL_GEN = "skt/kogpt2-base-v2"                                # 한국어 GPT-2

print("CLS:", MODEL_CLS)
print("QA :", MODEL_QA)
print("MT :", MODEL_MT_KOEN, "|", MODEL_MT_ENKO)
print("GEN:", MODEL_GEN)

CLS: nlptown/bert-base-multilingual-uncased-sentiment
QA : deepset/xlm-roberta-base-squad2
MT : Helsinki-NLP/opus-mt-ko-en | facebook/nllb-200-distilled-600M
GEN: skt/kogpt2-base-v2


## 4) 토크나이저 인스펙션 (BERT vs GPT-2)

In [4]:
test_text = "한국은행은 기준금리를 동결했다."

tok_bert = AutoTokenizer.from_pretrained("klue/bert-base")
tok_gpt2 = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")

print("=== BERT 토크나이즈 ===")
print(tok_bert.tokenize(test_text))
enc_bert = tok_bert(test_text, return_tensors="pt")
print("input_ids:", enc_bert["input_ids"][0][:20].tolist())

print("\n=== GPT-2 토크나이즈 ===")
print(tok_gpt2.tokenize(test_text))
enc_gpt2 = tok_gpt2(test_text, return_tensors="pt")
print("input_ids:", enc_gpt2["input_ids"][0][:20].tolist())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

=== BERT 토크나이즈 ===
['한국은행', '##은', '기준', '##금리', '##를', '동결', '##했', '##다', '.']
input_ids: [2, 10039, 2073, 3872, 10015, 2138, 10937, 2371, 2062, 18, 3]

=== GPT-2 토크나이즈 ===
['▁한국은행', '은', '▁기준', '금', '리를', '▁동', '결', '했다.']
input_ids: [30429, 8135, 10081, 6953, 9379, 9070, 6879, 13872]


## 5) 텍스트 분류 (문장 분류 / 감정·극성)

In [5]:
clf = pipeline(
    "text-classification",
    model=MODEL_CLS, tokenizer=MODEL_CLS,
    device_map="auto", truncation=True
)

texts = [
    "이 종목은 실적이 좋아서 상승할 것 같아.",
    "리스크가 너무 커 보여서 불안합니다.",
    "배당 정책이 좋아서 장기 보유하고 싶네요."
]
preds = clf(texts)
for t, p in zip(texts, preds):
    print(f"- {t}\n  -> {p}\n")

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


- 이 종목은 실적이 좋아서 상승할 것 같아.
  -> {'label': '3 stars', 'score': 0.45788106322288513}

- 리스크가 너무 커 보여서 불안합니다.
  -> {'label': '2 stars', 'score': 0.500135600566864}

- 배당 정책이 좋아서 장기 보유하고 싶네요.
  -> {'label': '4 stars', 'score': 0.4772607684135437}



## 6) 질의응답 (Extractive QA)

In [6]:
qa = pipeline(
    "question-answering",
    model=MODEL_QA, tokenizer=MODEL_QA,
    device_map="auto"
)

context = (
    "한국은행은 기준금리를 연 3.5%로 동결했다. "
    "물가 둔화와 경기 흐름을 균형적으로 고려한 결정이라고 밝혔다."
)
q = "기준금리는 얼마인가?"
out = qa({"question": q, "context": context})
print("Q:", q)
print("A:", out)

config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Q: 기준금리는 얼마인가?
A: {'score': 0.5220321994274855, 'start': 11, 'end': 19, 'answer': ' 연 3.5%로'}


## 7) 번역 (KO→EN / EN→KO)

In [11]:
mt_koen = pipeline(
    "translation",
    model=MODEL_MT_KOEN, tokenizer=MODEL_MT_KOEN,
    device_map="auto", max_length=256
)
mt_enko = pipeline(
    task="translation",
    model=MODEL_MT_ENKO,
    tokenizer=MODEL_MT_ENKO,
    device_map="auto",
    max_length=256,
    # NLLB는 소스/타겟 언어 코드를 명시해야 함
    src_lang="eng_Latn",
    tgt_lang="kor_Hang",
)
ko = "전기차 관련 종목들이 강세를 보였다."
en = mt_koen(ko)[0]["translation_text"]
print("KO→EN:", en)

en_src = "The central bank kept the base rate unchanged at 3.5 percent."
ko_back = mt_enko(en_src)[0]["translation_text"]
print("EN→KO:", ko_back)

Device set to use cpu
Device set to use cpu


KO→EN: Electrical cars and other timbers seemed to be strong.
EN→KO: 중앙은행은 기본 금리를 3.5%로 유지했습니다.


## 8) 텍스트 생성 (GPT-2: 디코더)

In [12]:
gen = pipeline(
    "text-generation",
    model=MODEL_GEN, tokenizer=MODEL_GEN,
    device_map="auto",
    do_sample=True, top_p=0.92, temperature=0.8,
    max_new_tokens=80
)

prompt = "요즘 주식시장은"
outs = gen(prompt, num_return_sequences=2)
for i, o in enumerate(outs, 1):
    print(f"[{i}] {o['generated_text']}")

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

Device set to use cpu


[1] 요즘 주식시장은 최근 두 달 동안 무려 8%나 급락했다.
지난달 23일에도 8% 급락했다.
주식시장이 반등하기 위해서는 시간이 필요할 것이라는 전망도 나온다.
이 같은 이유로 최근 증시는 코스피와 코스닥이 모두 저점을 찍고 반등에 성공했다는 분석이다.
그러나 최근 들어 미국 증시가 다시 하락세로 돌아서는 등 악재가 이어지고 있어 증시 역시 하락세로 돌아설 가능성이
[2] 요즘 주식시장은 지난해 7월 이후 5개월째 하락세를 기록하고 있다.
지난 4월 말 이후 15개월 만에 처음으로 약세로 돌아섰고 코스피 지수도 지난해 11월 말 이후 3개월 만에 처음으로 1500선이 깨졌다.
삼성전자 주가도 지난주 말 12만3000원대로 한 달 만에 17만원대로 떨어졌고 LG전자도 지난주 말 9500원까지 떨어지며 지난해 11월 중순 이후


## 9) 배치 추론 · 패딩/잘림 옵션

In [13]:
clf_batch = pipeline(
    "text-classification", model=MODEL_CLS, tokenizer=MODEL_CLS,
    device_map="auto", truncation=True, padding=True
)

batch_texts = [
    "매출과 영업이익이 모두 증가했습니다.",
    "가이던스가 하향되어서 실망스러워요.",
    "신제품 출시가 임박해 기대됩니다.",
    "환율 변동성이 확대되어 리스크가 커지고 있습니다."
]
print(clf_batch(batch_texts))

Device set to use cpu


[{'label': '1 star', 'score': 0.39172330498695374}, {'label': '1 star', 'score': 0.3427411913871765}, {'label': '5 stars', 'score': 0.4703539311885834}, {'label': '3 stars', 'score': 0.33903244137763977}]


## 10) 토크나이저 옵션 맛보기 (padding/truncation/max_length)

In [14]:
tok = AutoTokenizer.from_pretrained(MODEL_CLS)
sample = "이 문장은 약간 길게 늘려서 토크나이저의 잘림 옵션을 확인해 보려고 합니다. " * 3

enc = tok(sample, padding="max_length", truncation=True, max_length=32, return_tensors="pt")
print("input_ids.shape:", enc["input_ids"].shape)
print("attention_mask.sum:", int(enc["attention_mask"].sum()))
print("decode(head):", tok.decode(enc["input_ids"][0][:20]))

input_ids.shape: torch.Size([1, 32])
attention_mask.sum: 32
decode(head): [CLS] 이 문장은 약간 길게 늘려서 토크나이저


## 11) 팁/트러블슈팅

- `sentencepiece`가 필요한 모델은 설치되어야 함(위 셋업 셀에 포함).
- 긴 문맥에서 QA가 잘리지 않게 하려면 `max_length`, `doc_stride` 옵션을 조정.
- GPU가 없으면 속도가 느릴 수 있으니, 데모 입력은 짧게 유지.
- 모델/토크나이저 ID 불일치 경고가 뜨면 **동일 ID로 통일**(pipeline에 model=, tokenizer= 모두 지정).
- 재현성이 중요하면 `revision="<commit>"`로 모델 버전 고정.